In [1]:
# importing the libraries
import pandas as pd
import sys

In [2]:
path = "../Plaid_Data_Sandbox/transactions.csv"

In [3]:
user = pd.read_csv(path)

### The Classifier for Essential, Not Essential and Others

- Hardcoding them for now

- Modelling can only be done as we get more data, a lot of this needs more time and refinement to be implemented properly, this is as bootstrap as it gets. Not the best way to implement or write code!!

In [4]:
# Hardcoding essential and non essential
essential_categories = ['Payment', 'Transfer', 'Deposit', 'Credit Card']
non_essential_categories = ['Travel', 'Recreation', 'Gyms and Fitness Centers', 'Fast Food']

def classify_transaction(category):
    if not isinstance(category, (list, str)):
        return 'Unknown'
    
    if isinstance(category, str):
        try:
            category_list = eval(category)
        except:
            return 'Unknown'
    else:
        category_list = category
    
    is_essential = any(essential in category_list for essential in essential_categories)
    is_non_essential = any(non_essential in category_list for non_essential in non_essential_categories)
    
    if is_essential:
        return 'Essential'
    elif is_non_essential:
        return 'Non-Essential'
    else:
        return 'Other'

user['classification'] = user['category'].apply(lambda x: classify_transaction(x))

# print('classification' in user.columns)

### The Summarizer and Score Card 

In [5]:
# for more info read the readme

with open('financial_game_summary.txt', 'w') as f:
    original_stdout = sys.stdout
    sys.stdout = f

    user['date'] = pd.to_datetime(user['date'])
    user['month_year'] = user['date'].dt.to_period('M')

    grouped_data = user.groupby(['account_id', 'month_year'])

    financial_game_scores = {}

    for (account_id, month_year), group in grouped_data:
        
        essential_total = 0
        non_essential_total = 0
        other_total = 0
        
        for index, transaction in group.iterrows():
            classification = classify_transaction(transaction['category'])
            amount = transaction['amount']
            if classification == 'Essential':
                essential_total += amount
                print(f"Account ID: {account_id}, Transaction: {transaction['name']}, Amount: ${amount:.2f}, Category: Essential")
            elif classification == 'Non-Essential':
                non_essential_total += amount
                print(f"Account ID: {account_id}, Transaction: {transaction['name']}, Amount: ${amount:.2f}, Category: Non-Essential")
            else:
                other_total += amount
                print(f"Account ID: {account_id}, Transaction: {transaction['name']}, Amount: ${amount:.2f}, Category: Other")


        essential_total = group[group['classification'] == 'Essential']['amount'].sum()
        non_essential_total = group[group['classification'] == 'Non-Essential']['amount'].sum()

        total_spending = essential_total + non_essential_total
        assumed_total_income = total_spending / 0.8  # Assuming 80% of income is spent

        budget_essentials = assumed_total_income * 0.5
        budget_non_essentials = assumed_total_income * 0.3

        compliance_essentials = essential_total <= budget_essentials
        compliance_non_essentials = non_essential_total <= budget_non_essentials

        base_score = 60
        score_essentials = 4 if compliance_essentials else -4
        score_non_essentials = 2 if compliance_non_essentials else -2
        score_savings = 10 # Assuming 10% is savings

        print(f"Account ID: {account_id}, Month-Year: {month_year}")
        print(f"Total spending on Essentials: ${essential_total:.2f}")
        print(f"Total spending on Non-Essentials: ${non_essential_total:.2f}")
        print(f"Budget for Essentials: ${budget_essentials:.2f}")
        print(f"Budget for Non-Essentials: ${budget_non_essentials:.2f}")

        print(f"Compliance with Essential budget: {'Yes' if compliance_essentials else 'No'}")
        print(f"Compliance with Non-Essential budget: {'Yes' if compliance_non_essentials else 'No'}")

        print("Base Score: ", base_score)
        print("Score from Essentials Budget Compliance: ", score_essentials)
        print("Score from Non-Essentials Budget Compliance: ", score_non_essentials)
        print("Score from Savings: ", score_savings)

        raw_score = base_score + score_essentials + score_non_essentials + score_savings
        final_score = min(raw_score, 100)

        if score_essentials < 0:
            print(f"Account ID: {account_id} lost points for exceeding the essential budget.")
        if score_non_essentials < 0:
            print(f"Account ID: {account_id} lost points for exceeding the non-essential budget.")
        if raw_score > 100:
            print(f"Account ID: {account_id} could have a score higher than 100, but it's capped at 100.")

        print(f"Final Score for the month: {final_score}\n")

        financial_game_scores[(account_id, month_year)] = {
            "Assumed Total Income": assumed_total_income,
            "Budget for Essentials": budget_essentials,
            "Budget for Non-Essentials": budget_non_essentials,
            "Essential Spending": essential_total,
            "Non-Essential Spending": non_essential_total,
            "Final Financial Game Score": final_score
        }

    financial_game_scores_df = pd.DataFrame.from_dict(financial_game_scores, orient='index')

    financial_game_scores_df.reset_index(inplace=True)

    financial_game_scores_df.rename(columns={'level_0': 'Account ID', 'level_1': 'Month-Year'}, inplace=True)

    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    print(financial_game_scores_df)


    sys.stdout = original_stdout